---
title: Creating and populating tables
tasl:
    source: fa2023\lectures\lect14-sql-create-insert-delete-update.qmd
    tags:
    - lecture
    - lecture-14
    topic: Creating and populating tables
---


# Creating and populating tables

## Data Definition Language (DDL)

:::: {.columns}
::: {.column width=50%}
### CRUD operations

* **C**reate 
    - tables using CREATE TABLE
    - records using INSERT INTO

* **R**etrive data from tables using SELECT

* **U**pdate

    - Tables using ALTER TABLE
    - Records using UPDATE

* **D**elete
    - tables using DROP TABLE
    - records using DELETE

:::
::: {.column width=39%}
### General workflow

There are many approaches to database creation.  For this class I recommend the following workflow:

* Create database then create tables and then insert data all at one time
* Store all commands in single SQL file
* Paste commands to a command line in one chunk
* Design file for reuse / rebuilding
* SQL executes commands from top to bottom in file, like an interpreter.

:::
::::


## SQL Creating tables

:::: {.columns}
::: {.column width=50%}
### SQL Tables

*Tables* correspond to *relations*.  

*Tables* are the fundamental object in a database.

*Tables* are created using the CREATE TABLE command.  As you can imagine, there
are plenty of options.

See the [MySQL CREATE TABLE documentation](https://dev.mysql.com/doc/refman/8.0/en/create-table.html)

:::
::: {.column width=39%}
### CREATE TABLE SYNTAX

    CREATE TABLE <table name> (
        [columns and data types],
        [integrity constraints],
        [lots of other goodies!]
    )

:::
::::

## CREATE TABLE example

:::: {.columns}
::: {.column width=48%}
### [CREATE TABLE](https://dev.mysql.com/doc/refman/8.0/en/create-table.html)

```sql
CREATE TABLE instructor (
  ID		int AUTO_INCREMENT,
  full_name	varchar(20) NOT NULL,
  dept_name	varchar(30),
  salary    decimal(10,2) CHECK (salary > 0),
  office    char(5) NOT NULL UNIQUE,
  PRIMARY KEY (ID),
  FOREIGN KEY (dept_name)
    REFERENCES departments (department_name)
);
```
*indentation and capitalization doesn't matter*

:::
::: {.column width=30%}
### Features

* columns, data types
* column constraints
    - AUTO_INCREMENT
    - NOT NULL
    - CHECK
    - UNIQUE
* table constraints
    - PRIMARY KEY
    - FOREIGN KEY

:::
::: {.column width=22%}
### [Data Types](https://dev.mysql.com/doc/refman/8.0/en/data-types.html)

* Numeric types
* Date and time
* String
* Spatial (POINT, LINE, etc.)
* JSON (structure)

:::
::::

## ALTER and DROP tables

:::: {.columns}
::: {.column width=50%}

### ALTER TABLE

```{sql}
ALTER TABLE tbl_name
    [alter_option [, alter_option] ...]
    [partition_options]
```
<p>&nbsp;</p>

* Alter options:

    - ADD columns or constraints
    - ALTER columns or constraints
    - DROP columns or constraints

* Partition options:
    - Related to storage organization

See the [MySQL ALTER TABLE documentation](https://dev.mysql.com/doc/refman/8.0/en/alter-table.html)

:::
::: {.column width=49%}
### DROP TABLE

```{sql}
DROP [TEMPORARY] TABLE [IF EXISTS]
    tbl_name [, tbl_name] ...
    [RESTRICT | CASCADE]
```
<p>&nbsp;</p>

* Can drop multiple tables with a single command.
* IF EXISTS modifier will drop only if the table exists.  This is handy
for eliminating errors in code.
* RESTRICT and CASCADE are useful when there are keys.
* DDL tip: put all drops at top of file.


See the [MySQL DROP TABLE documentation](https://dev.mysql.com/doc/refman/8.0/en/drop-table.html)

:::
::::

## Complete example

:::: {.columns}
::: {.column width=45%}
### Crows Foot Diagram

```{mermaid}
%%| echo: false
erDiagram
    CITY {
        int city_id PK
        string name
        string state_abbreviation FK
    }
    STATE {
        string state_abbreviation PK
        string name
        int country_id FK
        int capital_city_id FK
    }
    COUNTRY {
        int country_id PK
        string name
        int capital_city_id FK
    }
    COUNTRY ||--|{ STATE : "Has"
    STATE ||--|{ CITY : "Has"
    CITY ||--o| STATE : "Is capital of"
    CITY ||--o| COUNTRY : "Is capital of"
```
:::
::: {.column width=54%}
### SQL CREATE
```{sql}

SET foreign_key_checks = 0;
DROP TABLE IF EXISTS city;
DROP TABLE IF EXISTS state;
DROP TABLE IF EXISTS country;
SET foreign_key_checks = 1;

CREATE TABLE city (
    city_id int,
    city_name varchar(30),
    city_home_state varchar(5),
    PRIMARY KEY (city_id)
);

CREATE TABLE state (
    state_abbr varchar(5),
    state_name varchar(30),
    state_home_country_id int,
    state_capital_city_id int,
    PRIMARY KEY (state_abbr)
);

CREATE TABLE country (
    country_id int not null,
    country_name varchar(50) not null,
    country_capital_city_id int not null,
    PRIMARY KEY (country_id)
);

ALTER TABLE city ADD FOREIGN KEY (city_home_state) REFERENCES state ( state_abbr );
ALTER TABLE state ADD FOREIGN KEY (state_home_country_id) REFERENCES country (country_id);
ALTER TABLE state ADD FOREIGN KEY (state_capital_city_id) REFERENCES city (city_id);
ALTER TABLE country ADD FOREIGN KEY (country_capital_city_id) REFERENCES city (city_id);

```
:::
::::

## Inserting data into tables

:::: {.columns}
::: {.column width=65%}

### INSERT INTO syntax

```{sql}
INSERT INTO relation VALUES (V1, …, VN);
# Inserts a new row with all the values matching the order of the columns.
# BAD IN PRACTICE!
# Example:
INSERT INTO instructor VALUES (123, 'ALBERTO CANO', NULL, 1234.56, 'E4251');

INSERT INTO relation (Ci, …, Cj) VALUES (Vi, …, Vj);
# Inserts a new row with the values matching any order of the columns specified,
# any other column not listed is assumed to be NULL.
# Examples:

INSERT INTO instructor (ID, name, salary, office)
		      VALUES (123, 'ALBERTO CANO', 1234.56, 'E4251');

INSERT INTO instructor (name, salary, office)
		      VALUES ('ALBERTO CANO', 9876.5, 'W0465');	-- ID AUTO GENERATED

```
<p>&nbsp;</p>

See the [MySQL INSERT INTO documentation](https://dev.mysql.com/doc/refman/8.0/en/insert.html
)

:::
::: {.column width=35%}
### Features

* Bulk INSERT Can be used to insert multiple records at once.
* Lots of flexiblity in variable and value lists.
* Some DBMS can load CSV files directly. Otherwise a CSV file is converted into SQL INSERT commands and then SQL is run.
* Quarto strategy:
    1. Load CSV in python dataframe using Pandas, then
    1. Create table using Pandas.

:::
::::

## UPDATE data in existing tables

:::: {.columns}
::: {.column width=45%}

### UPDATE syntax

```{sql}
UPDATE [LOW_PRIORITY] [IGNORE] table_reference
    SET assignment_list
    [WHERE where_condition]
    [ORDER BY ...]
    [LIMIT row_count]

value:
    {expr | DEFAULT}

assignment:
    col_name = value

assignment_list:
    assignment [, assignment] ...
```
<p>&nbsp;</p>

See the [MySQL UPDATE docs](https://dev.mysql.com/doc/refman/8.0/en/update.html)

:::
::: {.column width=55%}
### UPDATE examples
```{sql}
UPDATE instructor SET salary = 4321.00 WHERE ID = 123;
# SAFE. Will update the salary based on the PRIMARY KEY.

UPDATE instructor SET salary = 4321.00 
WHERE name = 'ALBERTO CANO';
# DANGER! Will update the salary for ALL 
# instructors with the given name.

UPDATE instructor SET salary = 4321.00;
# DANGER! Will update everyone’s salary!

UPDATE instructor SET salary = salary * 1.05;
# DANGER! Will update everyone’s 
# salary +5% respectively.

```

:::
::::

## Deleting data from tables

:::: {.columns}
::: {.column width=50%}

### DELETE

 Opposite of SELECT with many common arguments.
```{sql}
DELETE [LOW_PRIORITY] [QUICK] [IGNORE]
  FROM tbl_name [[AS] tbl_alias]
  [PARTITION (partition_name [, partition_name] ...)]
  [WHERE where_condition]
  [ORDER BY ...]
  [LIMIT row_count]

# Examples

DELETE FROM instructor
 WHERE last_name like 'LEON%';

DELETE FROM employee where id=101;
```
<p>&nbsp;</p>

See the [MySQL DELETE statement docs](https://dev.mysql.com/doc/refman/8.0/en/delete.html)


:::
::: {.column width=50%}

### TRUNCATE

Removes all rows in a table and resets metadata (counters, indexes, etc)

```{sql}
TRUNCATE TABLE instructor;

```
<p>&nbsp;</p>

See the [MySQL TRUNCATE docs ](https://dev.mysql.com/doc/refman/8.0/en/truncate-table.html)


<p>&nbsp;</p>

### DROP TABLE

Removes table from the database.

```{sql}
DROP TABLE instructor;
DROP TABLE IF EXISTS country;

```
<p>&nbsp;</p>

See the [MySQL DROP TABLE docs](https://dev.mysql.com/doc/refman/8.0/en/drop-table.html)
)

:::
::::

## Large DDL example

```{sql}
CREATE TABLE job_grades (
grade_level VARCHAR(3),
lowest_sal  FLOAT,
highest_sal FLOAT,
PRIMARY KEY(grade_level)
);

INSERT INTO job_grades VALUES ('A', 1000, 2999);
INSERT INTO job_grades VALUES ('B', 3000, 5999);
INSERT INTO job_grades VALUES ('C', 6000, 9999);
INSERT INTO job_grades VALUES ('D', 10000, 14999);
INSERT INTO job_grades VALUES ('E', 15000, 24999);
INSERT INTO job_grades VALUES ('F', 25000, 40000);

CREATE TABLE regions (
region_id   INT,
region_name VARCHAR(25),
PRIMARY KEY(region_id)
);

INSERT INTO regions VALUES (1, 'Europe');
INSERT INTO regions VALUES (2, 'Americas');
INSERT INTO regions VALUES (3, 'Asia');
INSERT INTO regions VALUES (4, 'Middle East and Africa');

CREATE TABLE countries (
country_id      CHAR(2),
country_name    VARCHAR(40),
region_id       INT,
PRIMARY KEY (country_id),
FOREIGN KEY (region_id) REFERENCES regions(region_id) 
);

INSERT INTO countries VALUES ('IT','Italy', 1); 
INSERT INTO countries VALUES ('JP','Japan', 3); 
INSERT INTO countries VALUES ('US','United States of America', 2); 
INSERT INTO countries VALUES ('CA','Canada', 2); 
INSERT INTO countries VALUES ('CN','China', 3); 
INSERT INTO countries VALUES ('IN','India', 3); 
INSERT INTO countries VALUES ('AU','Australia', 3); 
INSERT INTO countries VALUES ('ZW','Zimbabwe', 4); 
INSERT INTO countries VALUES ('SG','Singapore', 3); 
INSERT INTO countries VALUES ('UK','United Kingdom', 1); 
INSERT INTO countries VALUES ('FR','France', 1); 
INSERT INTO countries VALUES ('DE','Germany', 1); 
INSERT INTO countries VALUES ('ZM','Zambia', 4); 
INSERT INTO countries VALUES ('EG','Egypt', 4); 
INSERT INTO countries VALUES ('BR','Brazil', 2); 
INSERT INTO countries VALUES ('CH','Switzerland', 1); 
INSERT INTO countries VALUES ('NL','Netherlands', 1); 
INSERT INTO countries VALUES ('MX','Mexico', 2); 
INSERT INTO countries VALUES ('KW','Kuwait', 4); 
INSERT INTO countries VALUES ('IL','Israel', 4); 
INSERT INTO countries VALUES ('DK','Denmark', 1); 
INSERT INTO countries VALUES ('ML','Malaysia', 3); 
INSERT INTO countries VALUES ('NG','Nigeria', 4); 
INSERT INTO countries VALUES ('AR','Argentina', 2); 
INSERT INTO countries VALUES ('BE','Belgium', 1);

CREATE TABLE locations (
location_id    INT(4),
street_address VARCHAR(40),
postal_code    VARCHAR(12),
city           VARCHAR(30) NOT NULL,
state_province VARCHAR(25),
country_id     CHAR(2),
PRIMARY KEY (location_id),
FOREIGN KEY (country_id) REFERENCES countries(country_id) 
);

INSERT INTO locations VALUES ( 1000 , '1297 Via Cola di Rie' , '00989' , 'Roma' , NULL , 'IT' );  
INSERT INTO locations VALUES ( 1100 , '93091 Calle della Testa' , '10934' , 'Venice' , NULL , 'IT' );  
INSERT INTO locations VALUES ( 1200 , '2017 Shinjuku-ku' , '1689' , 'Tokyo' , 'Tokyo Prefecture' , 'JP' );  
INSERT INTO locations VALUES ( 1300 , '9450 Kamiya-cho' , '6823' , 'Hiroshima' , NULL , 'JP' );  
INSERT INTO locations VALUES ( 1400 , '2014 Jabberwocky Rd' , '26192' , 'Southlake' , 'Texas' , 'US' );  
INSERT INTO locations VALUES ( 1500 , '2011 Interiors Blvd' , '99236' , 'South San Francisco' , 'California' , 'US' );  
INSERT INTO locations VALUES ( 1600 , '2007 Zagora St' , '50090' , 'South Brunswick' , 'New Jersey' , 'US' );  
INSERT INTO locations VALUES ( 1700 , '2004 Charade Rd' , '98199' , 'Seattle' , 'Washington' , 'US' );  
INSERT INTO locations VALUES ( 1800 , '147 Spadina Ave' , 'M5V 2L7' , 'Toronto' , 'Ontario' , 'CA' );  
INSERT INTO locations VALUES ( 1900 , '6092 Boxwood St' , 'YSW 9T2' , 'Whitehorse' , 'Yukon' , 'CA' );  
INSERT INTO locations VALUES ( 2000 , '40-5-12 Laogianggen' , '190518' , 'Beijing' , NULL , 'CN' );  
INSERT INTO locations VALUES ( 2100 , '1298 Vileparle (E)' , '490231' , 'Bombay' , 'Maharashtra' , 'IN' );  
INSERT INTO locations VALUES ( 2200 , '12-98 Victoria Street' , '2901' , 'Sydney' , 'New South Wales' , 'AU' );  
INSERT INTO locations VALUES ( 2300 , '198 Clementi North' , '540198' , 'Singapore' , NULL , 'SG' );  
INSERT INTO locations VALUES ( 2400 , '8204 Arthur St' , NULL , 'London' , NULL , 'UK' );  
INSERT INTO locations VALUES ( 2500 , 'Magdalen Centre, The Oxford Science Park' , 'OX9 9ZB' , 'Oxford' , 'Oxford' , 'UK' );  
INSERT INTO locations VALUES ( 2600 , '9702 Chester Road' , '09629850293' , 'Stretford' , 'Manchester' , 'UK' );  
INSERT INTO locations VALUES ( 2700 , 'Schwanthalerstr. 7031' , '80925' , 'Munich' , 'Bavaria' , 'DE' );  
INSERT INTO locations VALUES ( 2800 , 'Rua Frei Caneca 1360 ' , '01307-002' , 'Sao Paulo' , 'Sao Paulo' , 'BR' );  
INSERT INTO locations VALUES ( 2900 , '20 Rue des Corps-Saints' , '1730' , 'Geneva' , 'Geneve' , 'CH' );  
INSERT INTO locations VALUES ( 3000 , 'Murtenstrasse 921' , '3095' , 'Bern' , 'BE' , 'CH' );  
INSERT INTO locations VALUES ( 3100 , 'Pieter Breughelstraat 837' , '3029SK' , 'Utrecht' , 'Utrecht' , 'NL' );  
INSERT INTO locations VALUES ( 3200 , 'Mariano Escobedo 9991' , '11932' , 'Mexico City' , 'Distrito Federal,' , 'MX' );


CREATE TABLE departments (
department_id    INT(4),
department_name  VARCHAR(30) NOT NULL,
manager_id       INT(6),
location_id      INT(4),
PRIMARY KEY (department_id),
FOREIGN KEY (location_id) REFERENCES locations (location_id)
);

INSERT INTO departments VALUES ( 10 , 'Administration' , 200 , 1700 );
INSERT INTO departments VALUES ( 20 , 'Marketing' , 201 , 1800 );   
INSERT INTO departments VALUES ( 30 , 'Purchasing' , 114 , 1700	);  
INSERT INTO departments VALUES ( 40 , 'Human Resources' , 203 , 2400 );
INSERT INTO departments VALUES ( 50 , 'Shipping' , 121 , 1500 );  
INSERT INTO departments VALUES ( 60 , 'IT' , 103 , 1400 );  
INSERT INTO departments VALUES ( 70 , 'Public Relations' , 204 , 2700 );  
INSERT INTO departments VALUES ( 80 , 'Sales' , 145 , 2500 );  
INSERT INTO departments VALUES ( 90 , 'Executive' , 100 , 1700 );
INSERT INTO departments VALUES ( 100 , 'Finance' , 108 , 1700 );  
INSERT INTO departments VALUES ( 110 , 'Accounting' , 205 , 1700 );
INSERT INTO departments VALUES ( 120 , 'Treasury' , NULL , 1700 );
INSERT INTO departments VALUES ( 130 , 'Corporate Tax' , NULL , 1700 );
INSERT INTO departments VALUES ( 140 , 'Control And Credit' , NULL , 1700 );
INSERT INTO departments VALUES ( 150 , 'Shareholder Services' , NULL , 1700 );
INSERT INTO departments VALUES ( 160 , 'Benefits' , NULL , 1700 );
INSERT INTO departments VALUES ( 170 , 'Manufacturing' , NULL , 1700 );
INSERT INTO departments VALUES ( 180 , 'Construction' , NULL , 1700 );
INSERT INTO departments VALUES ( 190 , 'Contracting' , NULL , 1700 );
INSERT INTO departments VALUES ( 200 , 'Operations' , NULL , 1700 );
INSERT INTO departments VALUES ( 210 , 'IT Support' , NULL , 1700 );
INSERT INTO departments VALUES ( 220 , 'NOC' , NULL , 1700 );
INSERT INTO departments VALUES ( 230 , 'IT Helpdesk' , NULL , 1700 );
INSERT INTO departments VALUES ( 240 , 'Government Sales' , NULL , 1700 );
INSERT INTO departments VALUES ( 250 , 'Retail Sales' , NULL , 1700 );
INSERT INTO departments VALUES ( 260 , 'Recruiting' , NULL , 1700 );
INSERT INTO departments VALUES ( 270 , 'Payroll' , NULL , 1700 );


CREATE TABLE jobs (
job_id     VARCHAR(10),
job_title  VARCHAR(35) NOT NULL,
min_salary FLOAT,
max_salary FLOAT,
PRIMARY KEY(job_id)
);

INSERT INTO jobs VALUES ( 'AD_PRES', 'President', 20080, 40000);
INSERT INTO jobs VALUES ( 'AD_VP', 'Administration Vice President', 15000, 30000);
INSERT INTO jobs VALUES ( 'AD_ASST', 'Administration Assistant', 3000, 6000);
INSERT INTO jobs VALUES ( 'FI_MGR', 'Finance Manager', 8200, 16000);
INSERT INTO jobs VALUES ( 'FI_ACCOUNT', 'Accountant', 4200, 9000);
INSERT INTO jobs VALUES ( 'AC_MGR', 'Accounting Manager', 8200, 16000);
INSERT INTO jobs VALUES ( 'AC_ACCOUNT', 'Public Accountant', 4200, 9000);
INSERT INTO jobs VALUES ( 'SA_MAN', 'Sales Manager', 10000, 20080);
INSERT INTO jobs VALUES ( 'SA_REP', 'Sales Representative', 6000, 12008);
INSERT INTO jobs VALUES ( 'PU_MAN', 'Purchasing Manager', 8000, 15000);
INSERT INTO jobs VALUES ( 'PU_CLERK', 'Purchasing Clerk', 2500, 5500);
INSERT INTO jobs VALUES ( 'ST_MAN', 'Stock Manager', 5500, 8500);
INSERT INTO jobs VALUES ( 'ST_CLERK', 'Stock Clerk', 2008, 5000);
INSERT INTO jobs VALUES ( 'SH_CLERK', 'Shipping Clerk', 2500, 5500);
INSERT INTO jobs VALUES ( 'IT_PROG', 'Programmer', 4000, 10000);
INSERT INTO jobs VALUES ( 'MK_MAN', 'Marketing Manager', 9000, 15000);
INSERT INTO jobs VALUES ( 'MK_REP', 'Marketing Representative', 4000, 9000);
INSERT INTO jobs VALUES ( 'HR_REP', 'Human Resources Representative', 4000, 9000);
INSERT INTO jobs VALUES ( 'PR_REP', 'Public Relations Representative', 4500, 10500);

CREATE TABLE employees (
employee_id     INTEGER(6) AUTO_INCREMENT,
first_name      VARCHAR(20),
last_name       VARCHAR(25) NOT NULL,
email           VARCHAR(25) NOT NULL UNIQUE,
phone_number    VARCHAR(20),
hire_date       DATE NOT NULL,
job_id          VARCHAR(10) NOT NULL,
salary          DECIMAL(8,2) CHECK (salary > 0),
commission_pct  DECIMAL(2,2),
manager_id      INT(6),
department_id   INT(4),
PRIMARY KEY (employee_id),
FOREIGN KEY (department_id) REFERENCES departments (department_id),
FOREIGN KEY (job_id) REFERENCES jobs (job_id)
);

INSERT INTO employees VALUES ( 100, 'Steven', 'King', 'SKING', '515.123.4567', STR_TO_DATE('17-06-2003', '%d-%m-%Y'), 'AD_PRES', 24000, NULL, NULL, 90);
INSERT INTO employees VALUES ( 101, 'Neena', 'Kochhar', 'NKOCHHAR', '515.123.4568', STR_TO_DATE('21-09-2005', '%d-%m-%Y'), 'AD_VP', 17000, NULL, 100, 90);
INSERT INTO employees VALUES ( 102, 'Lex', 'De Haan', 'LDEHAAN', '515.123.4569', STR_TO_DATE('13-01-2001', '%d-%m-%Y'), 'AD_VP', 17000, NULL, 100, 90);
INSERT INTO employees VALUES ( 103, 'Alexander', 'Hunold', 'AHUNOLD', '590.423.4567', STR_TO_DATE('03-01-2006', '%d-%m-%Y'), 'IT_PROG', 9000, NULL, 102, 60);
INSERT INTO employees VALUES ( 104, 'Bruce', 'Ernst', 'BERNST', '590.423.4568', STR_TO_DATE('21-05-2007', '%d-%m-%Y'), 'IT_PROG', 6000, NULL, 103, 60);
INSERT INTO employees VALUES ( 105, 'David', 'Austin', 'DAUSTIN', '590.423.4569', STR_TO_DATE('25-06-2005', '%d-%m-%Y'), 'IT_PROG', 4800, NULL, 103, 60);
INSERT INTO employees VALUES ( 106, 'Valli', 'Pataballa', 'VPATABAL', '590.423.4560', STR_TO_DATE('05-02-2006', '%d-%m-%Y'), 'IT_PROG', 4800, NULL, 103, 60);
INSERT INTO employees VALUES ( 107, 'Diana', 'Lorentz', 'DLORENTZ', '590.423.5567', STR_TO_DATE('07-02-2007', '%d-%m-%Y'), 'IT_PROG', 4200, NULL, 103, 60);
INSERT INTO employees VALUES ( 108, 'Nancy', 'Greenberg', 'NGREENBE', '515.124.4569', STR_TO_DATE('17-08-2002', '%d-%m-%Y'), 'FI_MGR', 12008, NULL, 101, 100);
INSERT INTO employees VALUES ( 109, 'Daniel', 'Faviet', 'DFAVIET', '515.124.4169', STR_TO_DATE('16-08-2002', '%d-%m-%Y'), 'FI_ACCOUNT', 9000, NULL, 108, 100);
INSERT INTO employees VALUES ( 110, 'John', 'Chen', 'JCHEN', '515.124.4269', STR_TO_DATE('28-09-2005', '%d-%m-%Y'), 'FI_ACCOUNT', 8200, NULL, 108, 100);
INSERT INTO employees VALUES ( 111, 'Ismael', 'Sciarra', 'ISCIARRA', '515.124.4369', STR_TO_DATE('30-09-2005', '%d-%m-%Y'), 'FI_ACCOUNT', 7700, NULL, 108, 100);
INSERT INTO employees VALUES ( 112, 'Jose Manuel', 'Urman', 'JMURMAN', '515.124.4469', STR_TO_DATE('07-03-2006', '%d-%m-%Y'), 'FI_ACCOUNT', 7800, NULL, 108, 100);
INSERT INTO employees VALUES ( 113, 'Luis', 'Popp', 'LPOPP', '515.124.4567', STR_TO_DATE('07-12-2007', '%d-%m-%Y'), 'FI_ACCOUNT', 6900, NULL, 108, 100);
INSERT INTO employees VALUES ( 114, 'Den', 'Raphaely', 'DRAPHEAL', '515.127.4561', STR_TO_DATE('07-12-2002', '%d-%m-%Y'), 'PU_MAN', 11000, NULL, 100, 30);
INSERT INTO employees VALUES ( 115, 'Alexander', 'Khoo', 'AKHOO', '515.127.4562', STR_TO_DATE('18-05-2003', '%d-%m-%Y'), 'PU_CLERK', 3100, NULL, 114, 30);
INSERT INTO employees VALUES ( 116, 'Shelli', 'Baida', 'SBAIDA', '515.127.4563', STR_TO_DATE('24-12-2005', '%d-%m-%Y'), 'PU_CLERK', 2900, NULL, 114, 30);
INSERT INTO employees VALUES ( 117, 'Sigal', 'Tobias', 'STOBIAS', '515.127.4564', STR_TO_DATE('24-07-2005', '%d-%m-%Y'), 'PU_CLERK', 2800, NULL, 114, 30);
INSERT INTO employees VALUES ( 118, 'Guy', 'Himuro', 'GHIMURO', '515.127.4565', STR_TO_DATE('15-11-2006', '%d-%m-%Y'), 'PU_CLERK', 2600, NULL, 114, 30);
INSERT INTO employees VALUES ( 119, 'Karen', 'Colmenares', 'KCOLMENA', '515.127.4566', STR_TO_DATE('10-08-2007', '%d-%m-%Y'), 'PU_CLERK', 2500, NULL, 114, 30);
INSERT INTO employees VALUES ( 120, 'Matthew', 'Weiss', 'MWEISS', '650.123.1234', STR_TO_DATE('18-07-2004', '%d-%m-%Y'), 'ST_MAN', 8000, NULL, 100, 50);
INSERT INTO employees VALUES ( 121, 'Adam', 'Fripp', 'AFRIPP', '650.123.2234', STR_TO_DATE('10-04-2005', '%d-%m-%Y'), 'ST_MAN', 8200, NULL, 100, 50);
INSERT INTO employees VALUES ( 122, 'Payam', 'Kaufling', 'PKAUFLIN', '650.123.3234', STR_TO_DATE('01-05-2003', '%d-%m-%Y'), 'ST_MAN', 7900, NULL, 100, 50);
INSERT INTO employees VALUES ( 123, 'Shanta', 'Vollman', 'SVOLLMAN', '650.123.4234', STR_TO_DATE('10-10-2005', '%d-%m-%Y'), 'ST_MAN', 6500, NULL, 100, 50);
INSERT INTO employees VALUES ( 124, 'Kevin', 'Mourgos', 'KMOURGOS', '650.123.5234', STR_TO_DATE('16-11-2007', '%d-%m-%Y'), 'ST_MAN', 5800, NULL, 100, 50);
INSERT INTO employees VALUES ( 125, 'Julia', 'Nayer', 'JNAYER', '650.124.1214', STR_TO_DATE('16-07-2005', '%d-%m-%Y'), 'ST_CLERK', 3200, NULL, 120, 50);
INSERT INTO employees VALUES ( 126, 'Irene', 'Mikkilineni', 'IMIKKILI', '650.124.1224', STR_TO_DATE('28-09-2006', '%d-%m-%Y'), 'ST_CLERK', 2700, NULL, 120, 50);
INSERT INTO employees VALUES ( 127, 'James', 'Landry', 'JLANDRY', '650.124.1334', STR_TO_DATE('14-01-2007', '%d-%m-%Y'), 'ST_CLERK', 2400, NULL, 120, 50);
INSERT INTO employees VALUES ( 128, 'Steven', 'Markle', 'SMARKLE', '650.124.1434', STR_TO_DATE('08-03-2008', '%d-%m-%Y'), 'ST_CLERK', 2200, NULL, 120, 50);
INSERT INTO employees VALUES ( 129, 'Laura', 'Bissot', 'LBISSOT', '650.124.5234', STR_TO_DATE('20-08-2005', '%d-%m-%Y'), 'ST_CLERK', 3300, NULL, 121, 50);
INSERT INTO employees VALUES ( 130, 'Mozhe', 'Atkinson', 'MATKINSO', '650.124.6234', STR_TO_DATE('30-10-2005', '%d-%m-%Y'), 'ST_CLERK', 2800, NULL, 121, 50);
INSERT INTO employees VALUES ( 131, 'James', 'Marlow', 'JAMRLOW', '650.124.7234', STR_TO_DATE('16-02-2005', '%d-%m-%Y'), 'ST_CLERK', 2500, NULL, 121, 50);
INSERT INTO employees VALUES ( 132, 'TJ', 'Olson', 'TJOLSON', '650.124.8234', STR_TO_DATE('10-04-2007', '%d-%m-%Y'), 'ST_CLERK', 2100, NULL, 121, 50);
INSERT INTO employees VALUES ( 133, 'Jason', 'Mallin', 'JMALLIN', '650.127.1934', STR_TO_DATE('14-06-2004', '%d-%m-%Y'), 'ST_CLERK', 3300, NULL, 122, 50);
INSERT INTO employees VALUES ( 134, 'Michael', 'Rogers', 'MROGERS', '650.127.1834', STR_TO_DATE('26-08-2006', '%d-%m-%Y'), 'ST_CLERK', 2900, NULL, 122, 50);
INSERT INTO employees VALUES ( 135, 'Ki', 'Gee', 'KGEE', '650.127.1734', STR_TO_DATE('12-12-2007', '%d-%m-%Y'), 'ST_CLERK', 2400, NULL, 122, 50);
INSERT INTO employees VALUES ( 136, 'Hazel', 'Philtanker', 'HPHILTAN', '650.127.1634', STR_TO_DATE('06-02-2008', '%d-%m-%Y'), 'ST_CLERK', 2200, NULL, 122, 50);
INSERT INTO employees VALUES ( 137, 'Renske', 'Ladwig', 'RLADWIG', '650.121.1234', STR_TO_DATE('14-07-2003', '%d-%m-%Y'), 'ST_CLERK', 3600, NULL, 123, 50);
INSERT INTO employees VALUES ( 138, 'Stephen', 'Stiles', 'SSTILES', '650.121.2034', STR_TO_DATE('26-10-2005', '%d-%m-%Y'), 'ST_CLERK', 3200, NULL, 123, 50);
INSERT INTO employees VALUES ( 139, 'John', 'Seo', 'JSEO', '650.121.2019', STR_TO_DATE('12-02-2006', '%d-%m-%Y'), 'ST_CLERK', 2700, NULL, 123, 50);
INSERT INTO employees VALUES ( 140, 'Joshua', 'Patel', 'JPATEL', '650.121.1834', STR_TO_DATE('06-04-2006', '%d-%m-%Y'), 'ST_CLERK', 2500, NULL, 123, 50);
INSERT INTO employees VALUES ( 141, 'Trenna', 'Rajs', 'TRAJS', '650.121.8009', STR_TO_DATE('17-10-2003', '%d-%m-%Y'), 'ST_CLERK', 3500, NULL, 124, 50);
INSERT INTO employees VALUES ( 142, 'Curtis', 'Davies', 'CDAVIES', '650.121.2994', STR_TO_DATE('29-01-2005', '%d-%m-%Y'), 'ST_CLERK', 3100, NULL, 124, 50);
INSERT INTO employees VALUES ( 143, 'Randall', 'Matos', 'RMATOS', '650.121.2874', STR_TO_DATE('15-03-2006', '%d-%m-%Y'), 'ST_CLERK', 2600, NULL, 124, 50);
INSERT INTO employees VALUES ( 144, 'Peter', 'Vargas', 'PVARGAS', '650.121.2004', STR_TO_DATE('09-07-2006', '%d-%m-%Y'), 'ST_CLERK', 2500, NULL, 124, 50);
INSERT INTO employees VALUES ( 145, 'John', 'Russell', 'JRUSSEL', '011.44.1344.429268', STR_TO_DATE('01-10-2004', '%d-%m-%Y'), 'SA_MAN', 14000, .4, 100, 80);
INSERT INTO employees VALUES ( 146, 'Karen', 'Partners', 'KPARTNER', '011.44.1344.467268', STR_TO_DATE('05-01-2005', '%d-%m-%Y'), 'SA_MAN', 13500, .3, 100, 80);
INSERT INTO employees VALUES ( 147, 'Alberto', 'Errazuriz', 'AERRAZUR', '011.44.1344.429278', STR_TO_DATE('10-03-2005', '%d-%m-%Y'), 'SA_MAN', 12000, .3, 100, 80);
INSERT INTO employees VALUES ( 148, 'Gerald', 'Cambrault', 'GCAMBRAU', '011.44.1344.619268', STR_TO_DATE('15-10-2007', '%d-%m-%Y'), 'SA_MAN', 11000, .3, 100, 80);
INSERT INTO employees VALUES ( 149, 'Eleni', 'Zlotkey', 'EZLOTKEY', '011.44.1344.429018', STR_TO_DATE('29-01-2008', '%d-%m-%Y'), 'SA_MAN', 10500, .2, 100, 80);
INSERT INTO employees VALUES ( 150, 'Peter', 'Tucker', 'PTUCKER', '011.44.1344.129268', STR_TO_DATE('30-01-2005', '%d-%m-%Y'), 'SA_REP', 10000, .3, 145, 80);
INSERT INTO employees VALUES ( 151, 'David', 'Bernstein', 'DBERNSTE', '011.44.1344.345268', STR_TO_DATE('24-03-2005', '%d-%m-%Y'), 'SA_REP', 9500, .25, 145, 80);
INSERT INTO employees VALUES ( 152, 'Peter', 'Hall', 'PHALL', '011.44.1344.478968', STR_TO_DATE('20-08-2005', '%d-%m-%Y'), 'SA_REP', 9000, .25, 145, 80);
INSERT INTO employees VALUES ( 153, 'Christopher', 'Olsen', 'COLSEN', '011.44.1344.498718', STR_TO_DATE('30-03-2006', '%d-%m-%Y'), 'SA_REP', 8000, .2, 145, 80);
INSERT INTO employees VALUES ( 154, 'Nanette', 'Cambrault', 'NCAMBRAU', '011.44.1344.987668', STR_TO_DATE('09-12-2006', '%d-%m-%Y'), 'SA_REP', 7500, .2, 145, 80);
INSERT INTO employees VALUES ( 155, 'Oliver', 'Tuvault', 'OTUVAULT', '011.44.1344.486508', STR_TO_DATE('23-11-2007', '%d-%m-%Y'), 'SA_REP', 7000, .15, 145, 80);
INSERT INTO employees VALUES ( 156, 'Janette', 'King', 'JKING', '011.44.1345.429268', STR_TO_DATE('30-01-2004', '%d-%m-%Y'), 'SA_REP', 10000, .35, 146, 80);
INSERT INTO employees VALUES ( 157, 'Patrick', 'Sully', 'PSULLY', '011.44.1345.929268', STR_TO_DATE('04-03-2004', '%d-%m-%Y'), 'SA_REP', 9500, .35, 146, 80);
INSERT INTO employees VALUES ( 158, 'Allan', 'McEwen', 'AMCEWEN', '011.44.1345.829268', STR_TO_DATE('01-08-2004', '%d-%m-%Y'), 'SA_REP', 9000, .35, 146, 80);
INSERT INTO employees VALUES ( 159, 'Lindsey', 'Smith', 'LSMITH', '011.44.1345.729268', STR_TO_DATE('10-03-2005', '%d-%m-%Y'), 'SA_REP', 8000, .3, 146, 80);
INSERT INTO employees VALUES ( 160, 'Louise', 'Doran', 'LDORAN', '011.44.1345.629268', STR_TO_DATE('15-12-2005', '%d-%m-%Y'), 'SA_REP', 7500, .3, 146, 80);
INSERT INTO employees VALUES ( 161, 'Sarath', 'Sewall', 'SSEWALL', '011.44.1345.529268', STR_TO_DATE('03-11-2006', '%d-%m-%Y'), 'SA_REP', 7000, .25, 146, 80);
INSERT INTO employees VALUES ( 162, 'Clara', 'Vishney', 'CVISHNEY', '011.44.1346.129268', STR_TO_DATE('11-11-2005', '%d-%m-%Y'), 'SA_REP', 10500, .25, 147, 80);
INSERT INTO employees VALUES ( 163, 'Danielle', 'Greene', 'DGREENE', '011.44.1346.229268', STR_TO_DATE('19-03-2007', '%d-%m-%Y'), 'SA_REP', 9500, .15, 147, 80);
INSERT INTO employees VALUES ( 164, 'Mattea', 'Marvins', 'MMARVINS', '011.44.1346.329268', STR_TO_DATE('24-01-2008', '%d-%m-%Y'), 'SA_REP', 7200, .10, 147, 80);
INSERT INTO employees VALUES ( 165, 'David', 'Lee', 'DLEE', '011.44.1346.529268', STR_TO_DATE('23-02-2008', '%d-%m-%Y'), 'SA_REP', 6800, .1, 147, 80);
INSERT INTO employees VALUES ( 166, 'Sundar', 'Ande', 'SANDE', '011.44.1346.629268', STR_TO_DATE('24-03-2008', '%d-%m-%Y'), 'SA_REP', 6400, .10, 147, 80);
INSERT INTO employees VALUES ( 167, 'Amit', 'Banda', 'ABANDA', '011.44.1346.729268', STR_TO_DATE('21-04-2008', '%d-%m-%Y'), 'SA_REP', 6200, .10, 147, 80);
INSERT INTO employees VALUES ( 168, 'Lisa', 'Ozer', 'LOZER', '011.44.1343.929268', STR_TO_DATE('11-03-2005', '%d-%m-%Y'), 'SA_REP', 11500, .25, 148, 80);
INSERT INTO employees VALUES ( 169  , 'Harrison', 'Bloom', 'HBLOOM', '011.44.1343.829268', STR_TO_DATE('23-03-2006', '%d-%m-%Y'), 'SA_REP', 10000, .20, 148, 80);
INSERT INTO employees VALUES ( 170, 'Tayler', 'Fox', 'TFOX', '011.44.1343.729268', STR_TO_DATE('24-01-2006', '%d-%m-%Y'), 'SA_REP', 9600, .20, 148, 80);
INSERT INTO employees VALUES ( 171, 'William', 'Smith', 'WSMITH', '011.44.1343.629268', STR_TO_DATE('23-02-2007', '%d-%m-%Y'), 'SA_REP', 7400, .15, 148, 80);
INSERT INTO employees VALUES ( 172, 'Elizabeth', 'Bates', 'EBATES', '011.44.1343.529268', STR_TO_DATE('24-03-2007', '%d-%m-%Y'), 'SA_REP', 7300, .15, 148, 80);
INSERT INTO employees VALUES ( 173, 'Sundita', 'Kumar', 'SKUMAR', '011.44.1343.329268', STR_TO_DATE('21-04-2008', '%d-%m-%Y'), 'SA_REP', 6100, .10, 148, 80);
INSERT INTO employees VALUES ( 174, 'Ellen', 'Abel', 'EABEL', '011.44.1644.429267', STR_TO_DATE('11-05-2004', '%d-%m-%Y'), 'SA_REP', 11000, .30, 149, 80);
INSERT INTO employees VALUES ( 175, 'Alyssa', 'Hutton', 'AHUTTON', '011.44.1644.429266', STR_TO_DATE('19-03-2005', '%d-%m-%Y'), 'SA_REP', 8800, .25, 149, 80);
INSERT INTO employees VALUES ( 176, 'Jonathon', 'Taylor', 'JTAYLOR', '011.44.1644.429265', STR_TO_DATE('24-03-2006', '%d-%m-%Y'), 'SA_REP', 8600, .20, 149, 80);
INSERT INTO employees VALUES ( 177, 'Jack', 'Livingston', 'JLIVINGS', '011.44.1644.429264', STR_TO_DATE('23-04-2006', '%d-%m-%Y'), 'SA_REP', 8400, .20, 149, 80);
INSERT INTO employees VALUES ( 178, 'Kimberely', 'Grant', 'KGRANT', '011.44.1644.429263', STR_TO_DATE('24-05-2007', '%d-%m-%Y'), 'SA_REP', 7000, .15, 149, NULL);
INSERT INTO employees VALUES ( 179, 'Charles', 'Johnson', 'CJOHNSON', '011.44.1644.429262', STR_TO_DATE('04-01-2008', '%d-%m-%Y'), 'SA_REP', 6200, .10, 149, 80);
INSERT INTO employees VALUES ( 180, 'Winston', 'Taylor', 'WTAYLOR', '650.507.9876', STR_TO_DATE('24-01-2006', '%d-%m-%Y'), 'SH_CLERK', 3200, NULL, 120, 50);
INSERT INTO employees VALUES ( 181, 'Jean', 'Fleaur', 'JFLEAUR', '650.507.9877', STR_TO_DATE('23-02-2006', '%d-%m-%Y'), 'SH_CLERK', 3100, NULL, 120, 50);
INSERT INTO employees VALUES ( 182, 'Martha', 'Sullivan', 'MSULLIVA', '650.507.9878', STR_TO_DATE('21-06-2007', '%d-%m-%Y'), 'SH_CLERK', 2500, NULL, 120, 50);
INSERT INTO employees VALUES ( 183, 'Girard', 'Geoni', 'GGEONI', '650.507.9879', STR_TO_DATE('03-02-2008', '%d-%m-%Y'), 'SH_CLERK', 2800, NULL, 120, 50);
INSERT INTO employees VALUES ( 184, 'Nandita', 'Sarchand', 'NSARCHAN', '650.509.1876', STR_TO_DATE('27-01-2004', '%d-%m-%Y'), 'SH_CLERK', 4200, NULL, 121, 50);
INSERT INTO employees VALUES ( 185, 'Alexis', 'Bull', 'ABULL', '650.509.2876', STR_TO_DATE('20-02-2005', '%d-%m-%Y'), 'SH_CLERK', 4100, NULL, 121, 50);
INSERT INTO employees VALUES ( 186, 'Julia', 'Dellinger', 'JDELLING', '650.509.3876', STR_TO_DATE('24-06-2006', '%d-%m-%Y'), 'SH_CLERK', 3400, NULL, 121, 50);
INSERT INTO employees VALUES ( 187, 'Anthony', 'Cabrio', 'ACABRIO', '650.509.4876', STR_TO_DATE('07-02-2007', '%d-%m-%Y'), 'SH_CLERK', 3000, NULL, 121, 50);
INSERT INTO employees VALUES ( 188, 'Kelly', 'Chung', 'KCHUNG', '650.505.1876', STR_TO_DATE('14-06-2005', '%d-%m-%Y'), 'SH_CLERK', 3800, NULL, 122, 50);
INSERT INTO employees VALUES ( 189, 'Jennifer', 'Dilly', 'JDILLY', '650.505.2876', STR_TO_DATE('13-08-2005', '%d-%m-%Y'), 'SH_CLERK', 3600, NULL, 122, 50);
INSERT INTO employees VALUES ( 190, 'Timothy', 'Gates', 'TGATES', '650.505.3876', STR_TO_DATE('11-07-2006', '%d-%m-%Y'), 'SH_CLERK', 2900, NULL, 122, 50);
INSERT INTO employees VALUES ( 191, 'Randall', 'Perkins', 'RPERKINS', '650.505.4876', STR_TO_DATE('19-12-2007', '%d-%m-%Y'), 'SH_CLERK', 2500, NULL, 122, 50);
INSERT INTO employees VALUES ( 192, 'Sarah', 'Bell', 'SBELL', '650.501.1876', STR_TO_DATE('04-02-2004', '%d-%m-%Y'), 'SH_CLERK', 4000, NULL, 123, 50);
INSERT INTO employees VALUES ( 193, 'Britney', 'Everett', 'BEVERETT', '650.501.2876', STR_TO_DATE('03-03-2005', '%d-%m-%Y'), 'SH_CLERK', 3900, NULL, 123, 50);
INSERT INTO employees VALUES ( 194, 'Samuel', 'McCain', 'SMCCAIN', '650.501.3876', STR_TO_DATE('01-07-2006', '%d-%m-%Y'), 'SH_CLERK', 3200, NULL, 123, 50);
INSERT INTO employees VALUES ( 195, 'Vance', 'Jones', 'VJONES', '650.501.4876', STR_TO_DATE('17-03-2007', '%d-%m-%Y'), 'SH_CLERK', 2800, NULL, 123, 50);
INSERT INTO employees VALUES ( 196, 'Alana', 'Walsh', 'AWALSH', '650.507.9811', STR_TO_DATE('24-04-2006', '%d-%m-%Y'), 'SH_CLERK', 3100, NULL, 124, 50);
INSERT INTO employees VALUES ( 197, 'Kevin', 'Feeney', 'KFEENEY', '650.507.9822', STR_TO_DATE('23-05-2006', '%d-%m-%Y'), 'SH_CLERK', 3000, NULL, 124, 50);
INSERT INTO employees VALUES ( 198, 'Donald', 'OConnell', 'DOCONNEL', '650.507.9833', STR_TO_DATE('21-06-2007', '%d-%m-%Y'), 'SH_CLERK', 2600, NULL, 124, 50);
INSERT INTO employees VALUES ( 199, 'Douglas', 'Grant', 'DGRANT', '650.507.9844', STR_TO_DATE('13-01-2008', '%d-%m-%Y'), 'SH_CLERK', 2600, NULL, 124, 50);
INSERT INTO employees VALUES ( 200, 'Jennifer', 'Whalen', 'JWHALEN', '515.123.4444', STR_TO_DATE('17-09-2003', '%d-%m-%Y'), 'AD_ASST', 4400, NULL, 101, 10);
INSERT INTO employees VALUES ( 201, 'Michael', 'Hartstein', 'MHARTSTE', '515.123.5555', STR_TO_DATE('17-02-2004', '%d-%m-%Y'), 'MK_MAN', 13000, NULL, 100, 20);
INSERT INTO employees VALUES ( 202, 'Pat', 'Fay', 'PFAY', '603.123.6666', STR_TO_DATE('17-08-2005', '%d-%m-%Y'), 'MK_REP', 6000, NULL, 201, 20);
INSERT INTO employees VALUES ( 203, 'Susan', 'Mavris', 'SMAVRIS', '515.123.7777', STR_TO_DATE('07-06-2002', '%d-%m-%Y'), 'HR_REP', 6500, NULL, 101, 40);
INSERT INTO employees VALUES ( 204, 'Hermann', 'Baer', 'HBAER', '515.123.8888', STR_TO_DATE('07-06-2002', '%d-%m-%Y'), 'PR_REP', 10000, NULL, 101, 70);
INSERT INTO employees VALUES ( 205, 'Shelley', 'Higgins', 'SHIGGINS', '515.123.8080', STR_TO_DATE('07-06-2002', '%d-%m-%Y'), 'AC_MGR', 12008, NULL, 101, 110);
INSERT INTO employees VALUES ( 206, 'William', 'Gietz', 'WGIETZ', '515.123.8181', STR_TO_DATE('07-06-2002', '%d-%m-%Y'), 'AC_ACCOUNT', 8300, NULL, 205, 110);

ALTER TABLE departments ADD (CONSTRAINT dept_mgr_fk FOREIGN KEY (manager_id) REFERENCES employees (employee_id));

CREATE TABLE job_history (
employee_id   INT(6) NOT NULL,
start_date    DATE NOT NULL,
end_date      DATE NOT NULL,
job_id        VARCHAR(10) NOT NULL,
department_id INT(4),
PRIMARY KEY (employee_id, start_date, end_date),
FOREIGN KEY (job_id) REFERENCES jobs (job_id),
FOREIGN KEY (employee_id) REFERENCES employees (employee_id),
FOREIGN KEY (department_id) REFERENCES departments (department_id)
);

INSERT INTO job_history VALUES (102, STR_TO_DATE('13-01-2001', '%d-%m-%Y'), STR_TO_DATE('24-07-2006', '%d-%m-%Y'), 'IT_PROG', 60);
INSERT INTO job_history VALUES (101, STR_TO_DATE('21-09-1997', '%d-%m-%Y'), STR_TO_DATE('27-10-2001', '%d-%m-%Y'), 'AC_ACCOUNT', 110);
INSERT INTO job_history VALUES (101, STR_TO_DATE('28-10-2001', '%d-%m-%Y'), STR_TO_DATE('15-03-2005', '%d-%m-%Y'), 'AC_MGR', 110);
INSERT INTO job_history VALUES (201, STR_TO_DATE('17-02-2004', '%d-%m-%Y'), STR_TO_DATE('19-12-2007', '%d-%m-%Y'), 'MK_REP', 20);
INSERT INTO job_history VALUES  (114, STR_TO_DATE('24-03-2006', '%d-%m-%Y'), STR_TO_DATE('31-12-2007', '%d-%m-%Y'), 'ST_CLERK', 50);
INSERT INTO job_history VALUES  (122, STR_TO_DATE('01-01-2007', '%d-%m-%Y'), STR_TO_DATE('31-12-2007', '%d-%m-%Y'), 'ST_CLERK', 50);
INSERT INTO job_history VALUES  (200, STR_TO_DATE('17-09-1995', '%d-%m-%Y'), STR_TO_DATE('17-06-2001', '%d-%m-%Y'), 'AD_ASST', 90);
INSERT INTO job_history VALUES  (176, STR_TO_DATE('24-03-2006', '%d-%m-%Y'), STR_TO_DATE('31-12-2006', '%d-%m-%Y'), 'SA_REP', 80);
INSERT INTO job_history VALUES  (176, STR_TO_DATE('01-01-2007', '%d-%m-%Y'), STR_TO_DATE('31-12-2007', '%d-%m-%Y'), 'SA_MAN', 80);
INSERT INTO job_history VALUES  (200, STR_TO_DATE('01-07-2002', '%d-%m-%Y'), STR_TO_DATE('31-12-2006', '%d-%m-%Y'), 'AC_ACCOUNT', 90);

CREATE OR REPLACE VIEW emp_details_view (
   employee_id,
   job_id,
   manager_id,
   department_id,
   location_id,
   country_id,
   first_name,
   last_name,
   salary,
   commission_pct,
   department_name,
   job_title,
   city,
   state_province,
   country_name,
   region_name)
AS SELECT
  e.employee_id, 
  e.job_id, 
  e.manager_id, 
  e.department_id,
  d.location_id,
  l.country_id,
  e.first_name,
  e.last_name,
  e.salary,
  e.commission_pct,
  d.department_name,
  j.job_title,
  l.city,
  l.state_province,
  c.country_name,
  r.region_name
FROM
  employees e,
  departments d,
  jobs j,
  locations l,
  countries c,
  regions r
WHERE e.department_id = d.department_id
  AND d.location_id = l.location_id
  AND l.country_id = c.country_id
  AND c.region_id = r.region_id
  AND j.job_id = e.job_id 
;

CREATE INDEX emp_department_ix ON employees (department_id);

CREATE INDEX emp_job_ix ON employees (job_id);

CREATE INDEX emp_manager_ix ON employees (manager_id);

CREATE INDEX emp_name_ix ON employees (last_name, first_name);

CREATE INDEX dept_location_ix ON departments (location_id);

CREATE INDEX dept_name_ix ON departments (department_name);

CREATE INDEX jhist_job_ix ON job_history (job_id);

CREATE INDEX jhist_employee_ix ON job_history (employee_id);

CREATE INDEX jhist_department_ix ON job_history (department_id);

CREATE INDEX loc_city_ix ON locations (city);

CREATE INDEX loc_state_province_ix ON locations (state_province);

CREATE INDEX loc_country_ix ON locations (country_id);

DELIMITER //

CREATE PROCEDURE add_job_history
(
p_emp_id         INT(6),
p_start_date     DATE,
p_end_date       DATE,
p_job_id         VARCHAR(10),
p_department_id  INT(4)
)
BEGIN
  INSERT INTO job_history (employee_id, start_date, end_date, job_id, department_id) VALUES(p_emp_id, p_start_date, p_end_date, p_job_id, p_department_id);
END
//

DELIMITER //

CREATE TRIGGER update_job_history
  AFTER UPDATE ON employees
  FOR EACH ROW
BEGIN
  IF (new.job_id <> old.job_id OR new.department_id <> old.department_id) THEN
    CALL add_job_history(old.employee_id, old.hire_date, sysdate(), old.job_id, old.department_id);
  END IF;
END;
//
```

## DDL and DML summary

:::: {.columns}
::: {.column width=50%}
### DDL

* Data Definition Language
* Used to create databases and tables
* Used to define table, row and field constraints
* Operations generally done *atomically*, in
one big push.

### Preferred Workflow

* Create a file (e.g., MY-DDL.sql) and add
all the commands. 
* Test commands one-by-one in VS code.
* Test entire block to make sure it runs.
* Commands:
    CREATE TABLE, ALTER TABLE, bulk INSERT, DROP TABLE

:::
::: {.column width=50%}
### DML

* Data Manipulation Language
* Used to update individual records and fields
* Generally connected to Web/HTML front end
* Sometimes connected to API
* API is constructed separately from UI
* Data integrity paramount
* Code designed to run each operation as needed ad-hoc.
* Commands:
    INSERT INTO, UPDATE, DELETE

:::
::::
